<a href="https://colab.research.google.com/github/Bilal4801/DL-projects/blob/main/title_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Title Generator using LSTM

In [ ]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow.keras.utils as ku 


from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
with open('/content/drive/MyDrive/Titles.txt', encoding="UTF-8") as file:
    data = file.read()

In [ ]:
import string
def clean_text(data):
    """
    Removes non essential characters in corpus of text
    """
    data = "".join(v for v in data if v not in string.punctuation).lower()
    data = data.encode("utf8").decode("ascii",'ignore')
    return data

In [ ]:
cleaned = clean_text(data)
print(cleaned[:5])

sciki


In [ ]:
corpus = cleaned.split("\n")
print(corpus[:10])

['scikitlearn machine learning in python', 'pattern recognition and machine learning', 'gaussian processes in machine learning', 'machine learning in automated text categorization', 'machine learning', 'thumbs up sentiment classification using machine learning techniques', 'ensemble methods in machine learning', 'c4 5 programs for machine learning', 'uci machine learning repository', 'data mining practical machine learning tools and techniques']


In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(total_words)
inp_sequences[:10]

1840


[[161, 2],
 [161, 2, 1],
 [161, 2, 1, 7],
 [161, 2, 1, 7, 137],
 [162, 42],
 [162, 42, 4],
 [162, 42, 4, 2],
 [162, 42, 4, 2, 1],
 [138, 163],
 [138, 163, 7]]

In [ ]:
import numpy as np
import tensorflow.keras.utils as ku 

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
print(max_sequence_len)

21


In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

lstm_model = create_model(max_sequence_len, total_words)
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 10)            18400     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1840)              185840    
                                                                 
Total params: 248,640
Trainable params: 248,640
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
# lstm_model.save('/content/drive/MyDrive/title_model.h5')

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/title_model.h5')

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # print(token_list)

        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # print(token_list)
        predicted = model.predict(token_list, verbose=0) # y_predict = np.argmax(model.predict(x_test), axis=-1)
        # print(predicted)
        pre = np.argmax(predicted)
        # print(pre)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
        #   print(word, index)
          if index == predicted:
              output_word = word
              break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print(generate_text("", 5, lstm_model, max_sequence_len))
print(generate_text("euclidean", 4, lstm_model, max_sequence_len))
print(generate_text("generative", 5, model, max_sequence_len))
print(generate_text("ground breaking", 5, lstm_model, max_sequence_len))
print(generate_text("new", 4, lstm_model, max_sequence_len))
# print(generate_text("understanding", 5, lstm_model, max_sequence_len))
print(generate_text("long short term memory", 6, lstm_model, max_sequence_len))
print(generate_text("LSTM", 6, lstm_model, max_sequence_len))
print(generate_text("a", 5, lstm_model, max_sequence_len))
print(generate_text("anomaly", 5, lstm_model, max_sequence_len))
print(generate_text("data", 7, lstm_model, max_sequence_len))
print(generate_text("designing", 7, lstm_model, max_sequence_len))
print(generate_text("reinforcement", 7, lstm_model, max_sequence_len))

ValueError: ignored